In [116]:
import pandas as pd
import numpy as np
from datetime import datetime

In [117]:
 data_path='PriceData\MyPortRef.csv'
#data_path='PriceData\TestPort.csv'

trade_plan_path='MasterConfigData\ListTradePlan.xlsx'
trade_sheet='TestFundPhlips'

asset_comm_path='MasterConfigData\AssetInfo.xlsx'

result_path=f'ReportData\ReportTestPort_{trade_sheet}-ETF.xlsx'


In [118]:
# df=pd.read_csv('ASP-SME_KKP-SM-TRade.csv',index_col='Date/Time',parse_dates=['Date/Time'],dayfirst=True)
df=pd.read_csv(data_path,parse_dates=['Date/Time'],dayfirst=True)
# df.index.set_names('date',inplace=True)
df.columns=['symbol','date','price']

list_symbols=df['symbol'].unique()

# list_symbols=['ACWI','SPY']
#list_symbols=['KKP-SM-CAP','TISCOFLEXP','TSF-A']
print(list_symbols)
df.info()
print(df.tail())


['ACWI' 'AAXJ' 'BMSCITH.BK' 'BMSCG.BK' 'SPY' 'TDEX.BK']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1518 entries, 0 to 1517
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   symbol  1518 non-null   object        
 1   date    1518 non-null   datetime64[ns]
 2   price   1518 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 35.7+ KB
       symbol       date  price
1513  TDEX.BK 2023-01-05   9.96
1514  TDEX.BK 2023-01-06  10.00
1515  TDEX.BK 2023-01-09  10.07
1516  TDEX.BK 2023-01-10  10.07
1517  TDEX.BK 2023-01-12  10.01


In [119]:
dfPlan=pd.read_excel(trade_plan_path,engine="openpyxl",sheet_name=trade_sheet, \
                     parse_dates=['trade_date'],dtype ={'portion_pct': np.float64})



tradeDateList= [ d for d in list(dfPlan['trade_date']) ]


print(tradeDateList)
print("================================================================================================")

print(dfPlan)

print(f"No Buy/Sell {dfPlan.groupby(['action']).size()}")




[Timestamp('2022-07-20 00:00:00'), Timestamp('2022-07-27 00:00:00'), Timestamp('2022-08-25 00:00:00'), Timestamp('2022-08-30 00:00:00'), Timestamp('2022-10-21 00:00:00'), Timestamp('2022-11-04 00:00:00'), Timestamp('2022-11-10 00:00:00'), Timestamp('2022-12-09 00:00:00'), Timestamp('2022-12-15 00:00:00')]
  trade_date  portion_pct action
0 2022-07-20         50.0      b
1 2022-07-27         50.0      b
2 2022-08-25         50.0      s
3 2022-08-30         50.0      s
4 2022-10-21         40.0      b
5 2022-11-04         30.0      b
6 2022-11-10         30.0      b
7 2022-12-09         50.0      s
8 2022-12-15         50.0      s
No Buy/Sell action
b    5
s    4
dtype: int64


In [120]:
dfAssetInfo=pd.read_excel(asset_comm_path,engine="openpyxl",sheet_name='AssetInfo',\
                    index_col='symbol', dtype ={'buy_comm_pct': np.float64,'sell_comm_pct': np.float64})
dfAssetInfo=dfAssetInfo.loc[list(list_symbols),:]
print(dfAssetInfo)

            buy_comm_pct  sell_comm_pct
symbol                                 
ACWI                 0.0            0.0
AAXJ                 0.0            0.0
BMSCITH.BK           0.0            0.0
BMSCG.BK             0.0            0.0
SPY                  0.0            0.0
TDEX.BK              0.0            0.0


In [121]:
# check all symbol , there are the same trade (Most Important)


# veridf  there are both buy and sell if no sell
#we have to add last reord 100 % as of last dat in Amibroker database
buy_sum_act=dfPlan.query("action=='b'").portion_pct.sum()
sell_sum_act=dfPlan.query("action=='s'").portion_pct.sum()
all_portion=0
if buy_sum_act!=sell_sum_act:
    raise Exception("No.Action Buy and Sell  must be the same.")
else:
    all_portion=buy_sum_act   
    print(f"portion = {all_portion} on")
    


# day is ordering ? in plan

# verify sum buy =sum sell

# veriry all list in data price are in commision


portion = 200.0 on


In [122]:
TotalBudget=100000
rrr=TotalBudget % all_portion
if rrr !=0 :
  raise Exception("Budet % Portion is not xxxx")
else:
  print(rrr)

0.0


In [123]:
def set_commision_by_action(item):
  x=item['symbol']  
  if item['action']=='b':
    comm_type='buy_comm_pct'
  elif  item['action']=='s':
    comm_type='sell_comm_pct'
  try:
     return dfAssetInfo.loc[x,comm_type]
  except Exception as ex:
     print(f"Error : no {x} to get commission")
     return 0
def get_qty_buy(item):
    xCost=(item['portion_pct']/all_portion)*TotalBudget
    xPrice=item['price_on_comm']
    qty=xCost/xPrice
    return qty
def get_qty_sell(item,TotalQty):
    qty=(item['portion_pct']/all_portion)*TotalQty
    return qty





In [124]:
dict_dfTran={}
dfSummary=pd.DataFrame(columns=['Symbol','BuyValue','AvgBuyPrice','SellValue','AvgSellPrice','PL','Percent_PL'])
for symbol in list_symbols:
    
    
    dfBS=df[ (df['symbol']==symbol) & (df['date'].isin(tradeDateList))]
    noPlanedTrade= len(tradeDateList)
    noActualTrade=len(dfBS)
    if noPlanedTrade==len(dfBS):
    
        print(f"{symbol} Buy/Sell ={noPlanedTrade}")
        dfBS=dfBS.merge(dfPlan,how='inner',left_on='date',right_on='trade_date')

        dfBS['comm']=dfBS.apply(set_commision_by_action,axis=1)
        dfBS['price_on_comm']=dfBS.apply(lambda item : item['price']*(1+ (item['comm']/100)) if item['action']=='b' else item['price']*(1-(item['comm']/100)), axis=1)
        dfBS['price_on_comm']=dfBS['price_on_comm'].round(2)

        dfBS=dfBS.drop(columns=['trade_date'])
        #print(dfBS)
    
        print("================Buy=================")
        buyDF=dfBS.query("action=='b'")    
        # cal qury and actual value and  total_val and totl qury
        buyDF['qty']=buyDF.apply(get_qty_buy,axis=1)
        buyDF['value']=buyDF['qty']*buyDF['price_on_comm']
        buyDF['action_value']=buyDF['value']
        buyDF['action_qty']=buyDF['qty']
        print(buyDF[['date' ,'price_on_comm','qty','value','portion_pct']])

        TotalQty=buyDF['qty'].sum()
        TotalValue=round(buyDF['value'].sum(),2)
        AvgPrice=round(TotalValue/TotalQty,2)
        # print(f"Total Buy {symbol} : qty={TotalQty} and value={TotalValue} at AvgPrice={AvgPrice}")

        print("=================Sell=================")
        sellDF=dfBS.query("action=='s'")  
        sellDF['qty']=sellDF.apply(get_qty_sell,axis=1,args=(TotalQty,))
        sellDF['value']=sellDF['qty']*sellDF['price_on_comm']
        sellDF['action_value']=sellDF['value']*-1
        sellDF['action_qty']=sellDF['qty']*-1
        print(sellDF[['date' ,'price_on_comm','qty','value','portion_pct']])

        SellTotalQty=sellDF['qty'].sum()
        SellTotalValue=round(sellDF['value'].sum(),2)
        SellAvgPrice=round(SellTotalValue/SellTotalQty,2)

        # print(f"Total Sell {symbol} : qty={SellTotalQty} and value={SellTotalValue} at AvgPrice={SellAvgPrice}")

        PL=round(SellTotalValue-TotalValue,0)
        PCT_PL=round(PL/TotalValue*100,2)

        summary_data={'Symbol':symbol,'BuyValue':TotalValue,'AvgBuyPrice':AvgPrice,'SellValue':SellTotalValue,'AvgSellPrice':SellAvgPrice, \
                      'PL':PL,'Percent_PL':PCT_PL
                     }
        dfSummary=dfSummary.append(summary_data,ignore_index=True)

        dfTran=pd.concat([buyDF,sellDF],axis=0)
        dfTran=dfTran.sort_values(by=['date'])
        dfTran[['value','action_value']]=dfTran[['value','action_value']].round(0)

        dict_dfTran[symbol]=dfTran
    else:
        print(f"{symbol} has been exclued from trading process due to{noActualTrade}<> {noPlanedTrade}")


ACWI Buy/Sell =9
================Buy=================
        date  price_on_comm         qty    value  portion_pct
0 2022-07-20          86.39  289.385346  25000.0         50.0
1 2022-07-27          87.85  284.575982  25000.0         50.0
4 2022-10-21          80.83  247.432884  20000.0         40.0
5 2022-11-04          82.33  182.193611  15000.0         30.0
6 2022-11-10          86.28  173.852573  15000.0         30.0
=================Sell=================
        date  price_on_comm         qty         value  portion_pct
2 2022-08-25          90.60  294.360099  26669.024952         50.0
3 2022-08-30          86.46  294.360099  25450.374143         50.0
7 2022-12-09          87.34  294.360099  25709.411030         50.0
8 2022-12-15          85.75  294.360099  25241.378473         50.0
AAXJ Buy/Sell =9
================Buy=================
        date  price_on_comm         qty    value  portion_pct
0 2022-07-20          67.81  368.677186  25000.0         50.0
1 2022-07-27          

C:\Users\pongthsa\AppData\Local\Temp\ipykernel_5876\3727569838.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buyDF['qty']=buyDF.apply(get_qty_buy,axis=1)
C:\Users\pongthsa\AppData\Local\Temp\ipykernel_5876\3727569838.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buyDF['value']=buyDF['qty']*buyDF['price_on_comm']
C:\Users\pongthsa\AppData\Local\Temp\ipykernel_5876\3727569838.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

In [125]:
for symbol,tranDF in dict_dfTran.items():
  tranDF['net_value']=tranDF['action_value'].cumsum(axis = 0) 
  tranDF['net_qty']=tranDF['action_qty'].cumsum(axis = 0)  
  tranDF['avg_price']=round( tranDF['net_value']/tranDF['net_qty'])
  tranDF=tranDF.sort_values(by=['date'])
  print(tranDF[['date' ,'action','price_on_comm','qty','net_qty','value','net_value','avg_price']])

        date action  price_on_comm         qty     net_qty    value  \
0 2022-07-20      b          86.39  289.385346  289.385346  25000.0   
1 2022-07-27      b          87.85  284.575982  573.961327  25000.0   
2 2022-08-25      s          90.60  294.360099  279.601229  26669.0   
3 2022-08-30      s          86.46  294.360099  -14.758870  25450.0   
4 2022-10-21      b          80.83  247.432884  232.674014  20000.0   
5 2022-11-04      b          82.33  182.193611  414.867625  15000.0   
6 2022-11-10      b          86.28  173.852573  588.720198  15000.0   
7 2022-12-09      s          87.34  294.360099  294.360099  25709.0   
8 2022-12-15      s          85.75  294.360099    0.000000  25241.0   

   net_value  avg_price  
0    25000.0       86.0  
1    50000.0       87.0  
2    23331.0       83.0  
3    -2119.0      144.0  
4    17881.0       77.0  
5    32881.0       79.0  
6    47881.0       81.0  
7    22172.0       75.0  
8    -3069.0       -inf  
        date action  price_on

In [126]:
writer=pd.ExcelWriter(result_path,engine='xlsxwriter') 
dfSummary=dfSummary.sort_values(by=['Percent_PL'],ascending=False)
dfSummary.to_excel(writer, sheet_name="Summary",index=False)

for symbol,tranDF in dict_dfTran.items():
 tranDF.to_excel(writer, sheet_name=symbol,index=False) 
 #print(tranDF)
writer.save()

writer.close()

print("Successfully")


C:\Users\pongthsa\AppData\Local\Temp\ipykernel_5876\1780546000.py:8: FutureWarning: save is not part of the public API, usage can give in unexpected results and will be removed in a future version
  writer.save()


Successfully


D:\ProgramData\Anaconda3\envs\myquant_exp\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [127]:
dfSummary

,Symbol,BuyValue,AvgBuyPrice,SellValue,AvgSellPrice,PL,Percent_PL
1,AAXJ,100000.0,63.41,105028.22,66.60,5028.0,5.03
3,BMSCG.BK,100000.0,10.60,104453.17,11.08,4453.0,4.45
0,ACWI,100000.0,84.93,103070.19,87.54,3070.0,3.07
4,SPY,100000.0,389.16,102825.06,400.16,2825.0,2.82
5,TDEX.BK,100000.0,9.50,102620.17,9.74,2620.0,2.62
2,BMSCITH.BK,100000.0,10.96,101662.02,11.14,1662.0,1.66
